# Chains in LangChain

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

In [2]:
import pandas as pd
df = pd.read_csv('Data.csv')
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


### LLMChain

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [4]:
llm = ChatOpenAI(temperature=0.9)

In [5]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that nakes {product}?"
)

In [6]:
chain = LLMChain(llm=llm, prompt=prompt)

In [7]:
product = "Full Stack web applications with the lastest technologies and powered by AI"
chain.run(product)

'TechFusion+ or AIStackTech'

### SimpleSequentialChain

In [8]:
from langchain.chains import SimpleSequentialChain

In [9]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
        a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [10]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company: {company_name}"
)

# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [11]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                             )

In [12]:
overall_simple_chain(product)



> Entering new SimpleSequentialChain chain...
TechStack AI Solutions
TechStack AI Solutions is a cutting-edge technology company that specializes in developing AI solutions for businesses to streamline and automate processes.

> Finished chain.


{'input': 'Full Stack web applications with the lastest technologies and powered by AI',
 'output': 'TechStack AI Solutions is a cutting-edge technology company that specializes in developing AI solutions for businesses to streamline and automate processes.'}

### SequentialChain

In [13]:
from langchain.chains import SequentialChain

In [15]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to English:"
    "\n\n{Review}"
)
# chain 1: input= Review, output= English review
chain_one = LLMChain(llm=llm, prompt=first_prompt,
                     output_key='English_Review'
                     )

In [16]:
second_prompt = ChatPromptTemplate.from_template(
    "Summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review, output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt,
                     output_key="summary"
                     )

In [17]:
# prompt template 3: Translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)

# chain 3: input= Review, output= Language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                       )

In [19]:
# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the follwing"
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)

# chain 4: input: summary, language, output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                      )

In [20]:
# Overall_chain: input= Review, output= English_Review, summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary", "followup_message"],
    verbose=True
)

In [21]:
review=df.Review[5]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English_Review': "I find the taste mediocre. The foam does not hold, it's strange. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?",
 'summary': 'The reviewer expresses dissatisfaction with the taste and foam of the product, suspecting that it might be an old batch or counterfeit compared to the same ones purchased in stores.',
 'followup_message': "Réponse de suivi: Nous sommes désolés d'apprendre que vous n'êtes pas satisfait du goût et de la mousse de notre produit. Nous tenons à vous assurer que nos produits sont authentiques et nous prenons la qualité très au sérieux. Il est possible qu'il s'agisse d'un lot ancien ou contrefait, ce qui n'est pas conforme à nos normes. Nous vous prions de nous excuser pour cette expérience et nous vous invitons à nous contacter dir